In [4]:
import cv2
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import chainer
from chainer import cuda, serializers, functions as F
import chainer.links as L
from chainer import Chain, Variable, cuda, optimizer, optimizers, serializers
import pickle
import os
from chainer.links import caffe

def load_list(path):
    tuples = []
    for line in open(path):
        pair = line.replace("\n","")#.strip().split()
        #print(pair)
        tuples.append(pair)
    return tuples

#print(load_list("train.txt"))

def pos_feat_diff(feat,b_po,af_po):#Position feature difference
    batch,ch,w,h = feat.data.shape
    b_po = F.broadcast_to(b_po,(batch,ch,w,h)) #(1,1,46,46) to (1,128,46,46)
    af_po = F.broadcast_to(af_po,(batch,ch,w,h))
    feat = feat - b_po + af_po
    return feat

def pos_feat_sum(po):#Position Features Summary  shape (19,320,320)
    po = F.sum(po,axis=0,keepdims=True)
    po = F.resize_images(F.expand_dims(po,axis=0),(46,46))
    return po #shape (1,1,46,46)

class MEVIN(chainer.Chain):
    insize = 320

    def __init__(self):
        super(MEVIN, self).__init__(
            mlp1_1 = chainer.ChainList(
                *[L.Linear(None,46*46)
                  for i in range(19)]),
            mlp1_2 = chainer.ChainList(
                *[L.Linear(None,4)
                  for i in range(19)]),
            mlp2_1 = chainer.ChainList(
                *[L.Linear(None,4)
                  for i in range(19*19)]),
            mlp2_2 = chainer.ChainList(
                *[L.Linear(None,2)
                  for i in range(19*19)]),
            mlp3_1 = L.Linear(None,46*46),
            mlp3_2 = L.Linear(None,46*46),
            conv1 = L.Convolution2D(in_channels=1, out_channels=19, ksize=3, stride=1, pad=1),
            conv1_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv1_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv2 = L.Convolution2D(in_channels=19, out_channels=19, ksize=3, stride=1, pad=1),
            conv2_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv2_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv3 = L.Convolution2D(in_channels=19, out_channels=19, ksize=3, stride=1, pad=1),
            conv3_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv3_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
        )

    def __call__(self, x1,x2,x3,x4):#Pred = [x1,x2,x3,x4]
        #print(x1.shape)
        h1 = self.Mix_pofe(x1,x2,x3)
        h2 = self.Mix_pofe(x2,x3,x4)
        
        #print(len(h1))
        h1 = self.Interaction(h1)
        h2 = self.Interaction(h2)
        
        h = self.Aggregator(h1,h2) # (1,1,42,42)
        
        return h
    
    
    def Mix_pofe(self,b_x,af_x,aff_x):        
        b_x_list = F.split_axis(b_x, 19, axis=0) #[(1,320,320),......]
        af_x_list = F.split_axis(af_x, 19, axis=0) #[(1,320,320),......]
        aff_x_list = F.split_axis(aff_x, 19, axis=0)
        
        afb_x = []
        for i in range(len(b_x_list)):
            aff_af = F.concat((af_x_list[i],aff_x_list[i]),axis=1)
            afb_x.append(F.expand_dims(F.concat((b_x_list[i],aff_af),axis=1),axis=0)) #(1,3,320,320)
            
        return afb_x
    
    def Interaction(self,afb_x): 
        F_mlp = []
        for i in range(len(afb_x)):
            img = F.resize_images(afb_x[i],(46,46))
            pred = F.leaky_relu(self.mlp1_1[i](img))#(1,3,46,46) >> (1,46*46)
            pred = F.leaky_relu(self.mlp1_2[i](pred))#(1,24*24) >> (1,4)
            F_mlp.append(pred)
        
        self_inter = []
        inter = 0
        for i in range(len(F_mlp)):
            for ii in range(len(F_mlp)):
                #print(i)
                #print(ii)
                if i == ii:
                    h = F.concat((F_mlp[i],F_mlp[ii]),axis=1)
                    h = F.leaky_relu(self.mlp2_1[i * 19 + ii](h)) #(1,4)
                    #print("chack",h)
                    self_inter.append(self.mlp2_2[i * 19 + ii](h)) #(1,2)
                elif i != ii:
                    h = F.concat((F_mlp[i],F_mlp[ii]),axis=1)
                    h = F.leaky_relu(self.mlp2_1[i * 19 + ii](h)) #(1,4)
                    #print("test",h)
                    inter += F.leaky_relu(self.mlp2_2[i * 19 + ii](h))#(1,2)
                                          
        for iii in range(len(self_inter)):
            self_inter[iii] += inter
                                          
        self_inter_add_inter = F.stack(self_inter, axis=1)
        
        return self_inter_add_inter #(1,19*2)
                    
    def Aggregator(self,si_add1,si_add2):
        h = F.concat((si_add1,si_add2),axis=1)#(19,2*2)
        h = F.leaky_relu(self.mlp3_1(h)) #(1,46*46)
        h = F.leaky_relu(self.mlp3_2(h)) #(1,46*46)
        h = F.reshape(h,(1,1,46,46))
        
        h2 = F.leaky_relu(self.conv1(h))
        h2 = F.leaky_relu(self.conv1_a(h2))    
        h2 = F.leaky_relu(self.conv1_b(h2))
        h3 = F.leaky_relu(self.conv2(h2))
        h3 = F.leaky_relu(self.conv2_a(h3))    
        h3 = F.leaky_relu(self.conv2_b(h3))    
        h4 = F.leaky_relu(self.conv2(h3 + h2))
        h4 = F.leaky_relu(self.conv2_a(h4))    
        h4 = F.leaky_relu(self.conv2_b(h4)) 
        
        return h,h4
print("OK")

OK


In [5]:
model = MEVIN()

# GPUのセット
FLAG_GPU = True # GPUを使用するかどうか
if FLAG_GPU: # numpyかcuda.cupyか
    xp = cuda.cupy
    cuda.get_device(0).use()
    model.to_gpu()
else:
    xp = np

opt_model = optimizers.Adam(alpha=0.0002, beta1=0.5)
opt_model.setup(model)
opt_model.add_hook(chainer.optimizer.WeightDecay(0.00001))

print("OK")

OK


In [6]:
batch = 1

train_list = load_list("train.txt")

def train_l(loop_n,batch=batch,train_list=train_list):
    print(loop_n)
    Preds = []
    for i in range(len(train_list)-1):        
        if i <= 3:
            with open('use_data_analis/'+ train_list[i] +'_po.pickle', mode='rb') as f:
                po = pickle.load(f)
                if FLAG_GPU:
                    po = cuda.to_gpu(po)
            Preds.append(po)
            #print(i)
            
        elif i > 3:
            loss = 0
            loss1 = 0
            loss2 = 0
            with open('use_data_analis/'+ train_list[i] +'_po.pickle', mode='rb') as f:
                po = pickle.load(f)
            #print("nam",len(Preds))
            Preds.pop(0)
            if FLAG_GPU:
                po = cuda.to_gpu(po)
            Preds.append(po)
            
            with open('use_data_analis/'+ train_list[i+1] +'_po.pickle', mode='rb') as f:
                label = pickle.load(f) 
            
            label1 = pos_feat_sum(label)
            label2 = F.resize_images(F.expand_dims(label,axis=0),(46,46))#pos_feat_sum(label)
            
            if FLAG_GPU:
                label1 = cuda.to_gpu(label1.data)
                label2 = cuda.to_gpu(label2.data)
            
            with chainer.using_config('train', 'True'):
                result = model(Preds[0],Preds[1],Preds[2],Preds[3])
                loss1 = F.mean_squared_error(result[0],label1)
                loss2 = F.mean_squared_error(result[1],label2)
                loss = loss1 + loss2
            
            print("frame",i,"loss",loss.data,"loss1",loss1.data,"loss2",loss2.data)
            model.zerograds()
            loss.backward()
            loss.unchain_backward()
            opt_model.update()
            
            if loop_n % 100 == 0:
                if FLAG_GPU:
                    reimg = cuda.to_cpu(pos_feat_sum(result[1][0]).data)
                    laimg = cuda.to_cpu(pos_feat_sum(label2[0]).data)
                else:
                    reimg = pos_feat_sum(result[1][0]).data
                    laimg = pos_feat_sum(label2[0]).data
                    
                cv2.imwrite('result/reimg'+str(i)+'.png', (reimg.transpose(0,2,3,1)[0])*256)
                cv2.imwrite('result/laimg'+str(i)+'.png', (laimg.transpose(0,2,3,1)[0])*256)
                
                chainer.serializers.save_npz("result/model"+str(loop_n)+".npz",model)     
                
for i in range(1001):
    train_l(i)

0
frame 4 loss 0.10874702781438828 loss1 0.10435998439788818 loss2 0.004387040622532368
frame 5 loss 0.020449377596378326 loss1 0.019792689010500908 loss2 0.0006566880038008094
frame 6 loss 0.018877217546105385 loss1 0.018304092809557915 loss2 0.0005731245619244874
frame 7 loss 0.018126105889678 loss1 0.017549557611346245 loss2 0.0005765479290857911
frame 8 loss 0.017432162538170815 loss1 0.016813455149531364 loss2 0.0006187072140164673
frame 9 loss 0.016282128170132637 loss1 0.015690509229898453 loss2 0.0005916187656112015
frame 10 loss 0.01825660653412342 loss1 0.01733863167464733 loss2 0.0009179757325910032
frame 11 loss 0.013977154158055782 loss1 0.01335091795772314 loss2 0.000626235967501998
frame 12 loss 0.01927880011498928 loss1 0.017987102270126343 loss2 0.0012916986597701907
frame 13 loss 0.014427722431719303 loss1 0.013853381387889385 loss2 0.0005743411602452397
frame 14 loss 0.015132298693060875 loss1 0.013728201389312744 loss2 0.001404097187332809
frame 15 loss 0.0114162964

frame 18 loss 0.0032044367399066687 loss1 0.0025918949395418167 loss2 0.000612541742157191
frame 19 loss 0.0038118164520710707 loss1 0.003297658869996667 loss2 0.0005141575238667428
frame 20 loss 0.003593960776925087 loss1 0.003067037323489785 loss2 0.0005269233952276409
frame 21 loss 0.0041745007038116455 loss1 0.003667352255433798 loss2 0.0005071482737548649
frame 22 loss 0.004408311098814011 loss1 0.003853518981486559 loss2 0.0005547923501580954
frame 23 loss 0.004982213489711285 loss1 0.004409283399581909 loss2 0.0005729299737140536
frame 24 loss 0.005182442720979452 loss1 0.004655606113374233 loss2 0.0005268367240205407
frame 25 loss 0.006058611907064915 loss1 0.005509921815246344 loss2 0.0005486899754032493
frame 26 loss 0.00564882205799222 loss1 0.005099678412079811 loss2 0.0005491435877047479
frame 27 loss 0.005650521256029606 loss1 0.00510628055781126 loss2 0.0005442409310489893
frame 28 loss 0.0050048730336129665 loss1 0.004460562951862812 loss2 0.0005443101399578154
frame 29

frame 31 loss 0.003205267246812582 loss1 0.0026778648607432842 loss2 0.000527402269653976
frame 32 loss 0.002492206636816263 loss1 0.0019610561430454254 loss2 0.0005311504937708378
frame 33 loss 0.0020582936704158783 loss1 0.0015464378520846367 loss2 0.0005118558183312416
frame 34 loss 0.0015149388927966356 loss1 0.001009796978905797 loss2 0.0005051418556831777
frame 35 loss 0.0016265211161226034 loss1 0.0011298226891085505 loss2 0.000496698368806392
frame 36 loss 0.0025087876711040735 loss1 0.0019961248617619276 loss2 0.000512662751134485
frame 37 loss 0.003901595249772072 loss1 0.0033863086719065905 loss2 0.0005152865778654814
frame 38 loss 0.004530293866991997 loss1 0.004026494920253754 loss2 0.0005037987139075994
frame 39 loss 0.005755245685577393 loss1 0.0052420529536902905 loss2 0.0005131927318871021
frame 40 loss 0.006571286823600531 loss1 0.006056637968868017 loss2 0.0005146490293554962
frame 41 loss 0.007101928815245628 loss1 0.006582264788448811 loss2 0.0005196642014198005
fr

frame 5 loss 0.010077331215143204 loss1 0.009563985280692577 loss2 0.0005133454687893391
frame 6 loss 0.005214294418692589 loss1 0.0047444249503314495 loss2 0.0004698695265688002
frame 7 loss 0.003939811140298843 loss1 0.0034679991658777 loss2 0.0004718122072517872
frame 8 loss 0.002754263347014785 loss1 0.0022680943366140127 loss2 0.0004861689521931112
frame 9 loss 0.0025747003965079784 loss1 0.0020950499456375837 loss2 0.0004796504508703947
frame 10 loss 0.004535098560154438 loss1 0.004039720166474581 loss2 0.0004953781608492136
frame 11 loss 0.0046013882383704185 loss1 0.004118054173886776 loss2 0.0004833340353798121
frame 12 loss 0.003732208162546158 loss1 0.0032444698736071587 loss2 0.00048773817252367735
frame 13 loss 0.002782862400636077 loss1 0.002303123939782381 loss2 0.00047973840264603496
frame 14 loss 0.0033422433771193027 loss1 0.002838711952790618 loss2 0.0005035314825363457
frame 15 loss 0.003689534729346633 loss1 0.0031996516045182943 loss2 0.0004898830666206777
frame 1

frame 18 loss 0.003726527327671647 loss1 0.0032277291174978018 loss2 0.0004987981519661844
frame 19 loss 0.0034896130673587322 loss1 0.003073451342061162 loss2 0.0004161618126090616
frame 20 loss 0.0031076313462108374 loss1 0.002677231328561902 loss2 0.00043039993033744395
frame 21 loss 0.003541412530466914 loss1 0.0031276470981538296 loss2 0.0004137654323130846
frame 22 loss 0.002590859541669488 loss1 0.002133328467607498 loss2 0.00045753110316582024
frame 23 loss 0.0027371488977223635 loss1 0.002261465648189187 loss2 0.0004756831913255155
frame 24 loss 0.002679414115846157 loss1 0.0022422366309911013 loss2 0.0004371775430627167
frame 25 loss 0.0032396779861301184 loss1 0.0027768192812800407 loss2 0.0004628586466424167
frame 26 loss 0.003075815737247467 loss1 0.002618759637698531 loss2 0.000457056041341275
frame 27 loss 0.0034451440442353487 loss1 0.002985773142427206 loss2 0.0004593708144966513
frame 28 loss 0.0030586603097617626 loss1 0.002602399094030261 loss2 0.0004562612739391625

...

frame 16 loss 0.00018912080849986523 loss1 9.986881195800379e-05 loss2 8.925199654186144e-05
frame 17 loss 0.00019053219875786453 loss1 0.0001050640712492168 loss2 8.546812750864774e-05
frame 18 loss 0.0001958375796675682 loss1 8.808737038634717e-05 loss2 0.00010775021655717865
frame 19 loss 0.00025739148259162903 loss1 0.00016562652308493853 loss2 9.176494495477527e-05
frame 20 loss 0.00030956792761571705 loss1 0.000197475659660995 loss2 0.00011209226795472205
frame 21 loss 0.0002736227761488408 loss1 0.0001603753335075453 loss2 0.00011324743536533788
frame 22 loss 0.00022720113338436931 loss1 0.00011602438462432474 loss2 0.00011117674876004457
frame 23 loss 0.00023616102407686412 loss1 0.0001244514569407329 loss2 0.00011170956713613123
frame 24 loss 0.00023437988420482725 loss1 0.0001464927918277681 loss2 8.788709237705916e-05
frame 25 loss 0.0001875402231235057 loss1 0.00010565382399363443 loss2 8.188640640582889e-05
frame 26 loss 0.0001702543522696942 loss1 7.705586176598445e-05 lo

frame 26 loss 0.00022201455431059003 loss1 0.00012370586046017706 loss2 9.830869385041296e-05
frame 27 loss 0.0002079927216982469 loss1 0.00011719633766915649 loss2 9.07963840290904e-05
frame 28 loss 0.0002097876713378355 loss1 0.00013013310672249645 loss2 7.965456461533904e-05
frame 29 loss 0.00023460612283088267 loss1 0.000154893466969952 loss2 7.971265586093068e-05
frame 30 loss 0.0002883657580241561 loss1 0.0002067847817670554 loss2 8.158098353305832e-05
frame 31 loss 0.00019308294577058405 loss1 0.00011232117685722187 loss2 8.076176891336218e-05
frame 32 loss 0.00017018362996168435 loss1 9.342323755845428e-05 loss2 7.676039967918769e-05
frame 33 loss 0.00018526919302530587 loss1 0.00011159611312905326 loss2 7.36730798962526e-05
frame 34 loss 0.00015128342784009874 loss1 8.082367276074365e-05 loss2 7.045974780339748e-05
frame 35 loss 0.00015624087245669216 loss1 8.789046842139214e-05 loss2 6.835040403530002e-05
frame 36 loss 0.0002452514017932117 loss1 0.0001719372667139396 loss2 7

frame 37 loss 0.00021555036073550582 loss1 0.00014611239021178335 loss2 6.943797779968008e-05
frame 38 loss 0.00019513966981321573 loss1 0.00012063908070558682 loss2 7.450059638358653e-05
frame 39 loss 0.00014124150038696826 loss1 6.266291165957227e-05 loss2 7.857858145143837e-05
frame 40 loss 0.00014134825323708355 loss1 6.22263178229332e-05 loss2 7.912193541415036e-05
frame 41 loss 0.00015878549311310053 loss1 7.58566238800995e-05 loss2 8.292886195704341e-05
frame 42 loss 0.00016147238784469664 loss1 7.932297739898786e-05 loss2 8.21494177216664e-05
655
frame 4 loss 0.00026787183014675975 loss1 0.00013406456855591387 loss2 0.0001338072761427611
frame 5 loss 0.0002057917881757021 loss1 9.644756937632337e-05 loss2 0.00010934422607533634
frame 6 loss 0.00022190657909959555 loss1 0.00014710456889588386 loss2 7.48020174796693e-05
frame 7 loss 0.00031732002389617264 loss1 0.00023164307640399784 loss2 8.567694021621719e-05
frame 8 loss 0.000279809843050316 loss1 0.00019726956088561565 loss2 

frame 9 loss 0.00016819563461467624 loss1 8.18311600596644e-05 loss2 8.636446727905422e-05
frame 10 loss 0.00018925535550806671 loss1 0.0001049942075042054 loss2 8.426114800386131e-05
frame 11 loss 0.00018673509475775063 loss1 9.850069181993604e-05 loss2 8.823441021377221e-05
frame 12 loss 0.00023318902822211385 loss1 0.00012680857616942376 loss2 0.0001063804593286477
frame 13 loss 0.0002481553819961846 loss1 0.00014324132644105703 loss2 0.00010491405555512756
frame 14 loss 0.00022224278654903173 loss1 0.00011590572103159502 loss2 0.00010633707279339433
frame 15 loss 0.0001959836226888001 loss1 9.840240090852603e-05 loss2 9.758122905623168e-05
frame 16 loss 0.00018343540432397276 loss1 9.172352292807773e-05 loss2 9.171188139589503e-05
frame 17 loss 0.00018093781545758247 loss1 9.298652003053576e-05 loss2 8.79512881510891e-05
frame 18 loss 0.00021368052694015205 loss1 0.00010914015001617372 loss2 0.00010454037692397833
frame 19 loss 0.00021429796470329165 loss1 0.00012443962623365223 lo

KeyboardInterrupt: 